In [1]:
#Import everything
import numpy as np
# from gym.utils import seeding
# from gym.spaces import Discrete, Tuple, Box
# import gym
from qiskit.quantum_info import state_fidelity
from qiskit import *
from numpy.linalg import matrix_power
import pandas as pd

In [2]:
#Set globally used variables
n = 10**6
k = 10

GATES = {
    0: np.array([[1, 1], [1, -1]]) * 1/np.sqrt(2), # H
    1: np.array([[1, 0], [0, np.exp(1j * np.pi / 4)]]), # T
    # 2: np.array([[0, 1], [1, 0]]), # X
    2: np.array([[1, 0], [0, 1]]) # I
}

thetas = np.array(pd.cut(np.linspace(0, np.pi, k), k, precision=10, include_lowest=True))
thetas[0] = pd.Interval(0, thetas[0].right, closed='both')
phis = np.array(pd.cut(np.linspace(0, 2*np.pi, 2*k), 2*k,  precision=10, include_lowest=True))
phis[0] = pd.Interval(0, phis[0].right, closed='both')

states = [(i, j) for i in range(len(thetas)) for j in range(len(phis))]
values = np.zeros(len(thetas) * len(phis))

In [3]:
def generate_target_circuit(n):
    s = np.array([1, 0])
    ht = GATES[0] @ GATES[1]
    return matrix_power(ht, n) @ s

def statevector_to_angles(state):
    svp = [abs(state[0])*np.exp(1j * np.angle(state[0])), abs(state[1])*np.exp(1j * np.angle(state[1]))]
    svp /= np.exp(1j * np.angle(state[0]))
    theta = 2 * np.arccos(abs(svp[0]))
    phi = np.angle(svp[1])
    if (phi < 0): phi += 2*np.pi
    return theta, phi
    # return np.cos(theta / 2) * np.array([1,0]) + np.exp(1j * phi) * np.sin(theta / 2) * np.array([0, 1])

def statevector_to_bloch_reg(state):
    theta, phi = statevector_to_angles(state)

    # take into consideration the poles
    for i in range(len(thetas)):
        if (theta in thetas[i]):
            theta_reg = i
    for i in range(len(phis)):
        if (phi in phis[i]):
            phi_reg = i

    if (theta_reg == 0):
        theta_reg = phi_reg = 0
    if (theta_reg == len(thetas)-1):
        theta_reg = len(thetas)-1
        phi_reg = len(phis)-1
    return (theta_reg, phi_reg)

def random_state_in_reg(reg):
    if (reg[0] == 0 or reg[0] == len(thetas)-1):
        phi = np.random.uniform(0, 2*np.pi)
    else:
        phi = np.random.uniform(phis[reg[1]].left, phis[reg[1]].right)
    theta = np.random.uniform(thetas[reg[0]].left, thetas[reg[0]].right)
    return np.cos(theta / 2) * np.array([1,0]) + np.exp(1j * phi) * np.sin(theta / 2) * np.array([0, 1])

def statevector_to_bloch_point(state):
    svp = [abs(state[0])*np.exp(1j * np.angle(state[0])), abs(state[1])*np.exp(1j * np.angle(state[1]))]
    svp /= np.exp(1j * np.angle(svp[0]))
    theta = 2 * np.arccos(abs(svp[0]))
    phi = np.angle(svp[1])
    return np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta)

def random_unitary(dim):
  # follows the algorithm in https://arxiv.org/pdf/math-ph/0609050.pdf
  Z = np.array([np.random.normal(0, 1) + np.random.normal(0, 1) * 1j for _ in range(dim ** 2)]).reshape(dim, dim)
  Q, R = np.linalg.qr(Z)
  diag = np.diagonal(R)
  lamb = np.diag(diag) / np.absolute(diag)
  unitary = np.matmul(Q, lamb)
  assert np.allclose(unitary.conj().T @ unitary, np.eye(dim))
  return unitary

In [4]:
goal = generate_target_circuit(n=n)
goal_region = statevector_to_bloch_reg(goal)


In [11]:
#This defines the reward function. Reward to network iff it is in the goal bloch region
def R(state):
    if (state == goal_region):
        return 1
        # if (action <= len(GATES) - 2):
        #     return 0
        # else:
        #     return 0.1 # to encourage using identity
    else:
        return 0

In [414]:
def run_episode(policy, max_length_of_episode):
    episode = []
    t = 0 # time t
    current_state = 0 # The index of a randomly chosen region
    while t < max_length_of_episode and current_state != states.index(goal_region):
        timestep = []
        timestep.append(current_state) # Add the current state to the timestep

        action = np.random.choice(3, p=policy[current_state]) # Choose a random action under the policy based on the policies distribution

        r_state = random_state_in_reg(states[current_state]) # Choose a random region inside the state. This simulates the probablistic distribution
        current_state = states.index(statevector_to_bloch_reg(GATES[action] @ r_state)) # Apply the action and get our next state

        timestep.append(action) # Add that action to the timestep
        timestep.append(R(states[current_state])) # Add the reward of our future state
        episode.append(timestep) # Append the timestep to the episode
        t += 1
    
    return episode
        
    

In [278]:
#This is the First-Vist MC prediction for estimating V == V_Pi
def FV_Monte_Carlo_Estimation(policy, discount_factor=0.8, episode_count=1000, max_time = 100):
    V = np.zeros(len(states))

    for i in range(episode_count):
        G = 0
        episode = run_episode(policy=policy, max_length_of_episode=max_time)
        Returns = np.zeros(len(states))
        for t in reversed(range(0, len(episode))):
            s_t, a_t, r_t = episode[t]
            G += r_t
            Returns[s_t] = G
        V += Returns

    for i in range(1, len(phis)):
        V[i] = V[0]
        V[len(thetas)*len(phis)-1-i] = V[len(thetas)*len(phis)-1]
    return np.array(V)

In [283]:
#This shows that in theory, it can calucate the max just as well as dynamics 
policy = np.ones([len(states), len(GATES)]) / len(GATES)
max = np.argmax(FV_Monte_Carlo_Estimation(policy, episode_count=50000, max_time=15))
print(max)

117


In [403]:
def on_policy_fv_mc_control(policy=None, eplison=0.01, gamma = 1, episode_count=10000):
    if not policy:
        policy = np.ones([len(states), len(GATES)]) / len(GATES)
    Q = [np.zeros(len(GATES), dtype=np.float16) for i in range(len(states))]
    returns = {}

    for _ in range(episode_count):
        G = 0
        episode = run_episode(policy, 100)

        for i in reversed(range(0 , len(episode))):
            s_t, a_t, r_t = episode[i]
            state_action = (s_t, a_t)
            G = gamma*G + r_t
            if not state_action in [(x[0], x[1]) for x in episode[0:i]]: #Make sure the current state_action is the first occurance
                if returns.get(state_action):
                    returns[state_action].append(G)
                else:
                    returns[state_action] = [G]

                Q[s_t][a_t] = sum(returns[state_action]) / len(returns[state_action])
                a_star = np.argmax(Q[s_t])
            
                for a in range(len(policy[s_t])):
                    if a == a_star:
                        policy[s_t][a] = 1 - eplison + (eplison / len(GATES))
                    else:
                        policy[s_t][a] = (eplison / len(GATES))
                
                
    return policy



In [418]:
n = 10**6
goal = generate_target_circuit(n=n)
goal_reg = statevector_to_bloch_reg(goal)

#Generate and train policy
policy = on_policy_fv_mc_control(episode_count=1000)
print(policy)


[[0.99333333 0.00333333 0.00333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.99333333 0.00333333 0.00333333]
 [0.99333333 0.00333333 0.00333333]
 [0.99333333 0.00333333 0.00333333]
 [0.99333333 0.00333333 0.00333333]
 [0.99333333 0.00333333 0.00333333]
 [0.99333333 0.00333333 0.00333333]
 [0.99333333 0.00333333 0.00333333]
 [0.99333333 0.00333333 0.00

In [419]:
optimal_programs = []
for i in range(k):
    converged = False
    while not converged:
        s = random_state_in_reg((0, 0))
        prog = []
        counter = 0
        while counter < 30:
            action = np.argmax(policy[states.index(statevector_to_bloch_reg(s))])
            next_s = GATES[action] @ s
            prog.append(action)
            # next_s = random_state_in_reg(statevector_to_bloch_reg(next_s))
            s = next_s
            counter += 1
            if (statevector_to_bloch_reg(s) == goal_reg):
                print('converged')
                converged = True
                break
        
    optimal_programs.append(prog)
optimal_programs

prog = optimal_programs[np.argmin(np.array([len(i) for i in optimal_programs]))]
fidelities = []
numTrials = 100
print(prog)
# prog = [0, 1, 1, 0]
for i in range(100000):
    s = np.array([1, 0])
    s = random_state_in_reg((0,0))
    for a in prog:
        s = GATES[a] @ s
    f = state_fidelity(s, goal)
    if (statevector_to_bloch_reg(s) == goal_reg):
        fidelities.append(f)
        break
    # print(goal, s)
print(np.average(fidelities))

KeyboardInterrupt: 